# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jan 4 2023 8:46AM,254242,10,PRF-44507-BO,Bio-PRF,Released
1,Jan 4 2023 8:43AM,254240,10,MSP217194,"Methapharm, Inc.",Released
2,Jan 4 2023 8:43AM,254240,10,MSP217195,"Methapharm, Inc.",Released
3,Jan 4 2023 8:43AM,254240,10,MSP217196,"Methapharm, Inc.",Released
4,Jan 4 2023 8:43AM,254240,10,MSP217197,"Methapharm, Inc.",Released
5,Jan 4 2023 8:43AM,254240,10,MSP217199,"Methapharm, Inc.",Released
6,Jan 4 2023 8:43AM,254240,10,MSP216468,"Methapharm, Inc.",Released
7,Jan 4 2023 8:37AM,254239,10,8985897,"Snap Medical Industries, LLC",Released
8,Jan 4 2023 8:35AM,254238,10,Enova-11159,Emerginnova,Released
9,Jan 4 2023 8:30AM,254196,10,0086280176,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
39,254237,Released,1
40,254238,Released,1
41,254239,Released,1
42,254240,Released,6
43,254242,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
254237,NaN,NaN,1.0
254238,NaN,NaN,1.0
254239,NaN,NaN,1.0
254240,NaN,NaN,6.0
254242,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254124,0.0,1.0,0.0
254136,0.0,0.0,45.0
254137,0.0,0.0,34.0
254138,0.0,0.0,37.0
254139,0.0,0.0,23.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
254124,0,1,0
254136,0,0,45
254137,0,0,34
254138,0,0,37
254139,0,0,23


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254124,0,1,0
1,254136,0,0,45
2,254137,0,0,34
3,254138,0,0,37
4,254139,0,0,23


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,254124,,1,
1,254136,,,45
2,254137,,,34
3,254138,,,37
4,254139,,,23


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jan 4 2023 8:46AM,254242,10,Bio-PRF
1,Jan 4 2023 8:43AM,254240,10,"Methapharm, Inc."
7,Jan 4 2023 8:37AM,254239,10,"Snap Medical Industries, LLC"
8,Jan 4 2023 8:35AM,254238,10,Emerginnova
9,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation
26,Jan 4 2023 8:26AM,254237,20,"HVL, LLC dba Atrium Innovations"
27,Jan 4 2023 8:25AM,254236,22,"NBTY Global, Inc."
28,Jan 4 2023 7:54AM,254235,19,"AdvaGen Pharma, Ltd"
29,Jan 3 2023 4:37PM,254230,15,"Person & Covey, Inc."
128,Jan 3 2023 4:22PM,254227,15,Lonza


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jan 4 2023 8:46AM,254242,10,Bio-PRF,,,1
1,Jan 4 2023 8:43AM,254240,10,"Methapharm, Inc.",,,6
2,Jan 4 2023 8:37AM,254239,10,"Snap Medical Industries, LLC",,,1
3,Jan 4 2023 8:35AM,254238,10,Emerginnova,,,1
4,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,,,17
5,Jan 4 2023 8:26AM,254237,20,"HVL, LLC dba Atrium Innovations",,,1
6,Jan 4 2023 8:25AM,254236,22,"NBTY Global, Inc.",,,1
7,Jan 4 2023 7:54AM,254235,19,"AdvaGen Pharma, Ltd",,,1
8,Jan 3 2023 4:37PM,254230,15,"Person & Covey, Inc.",,,99
9,Jan 3 2023 4:22PM,254227,15,Lonza,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:46AM,254242,10,Bio-PRF,1,,
1,Jan 4 2023 8:43AM,254240,10,"Methapharm, Inc.",6,,
2,Jan 4 2023 8:37AM,254239,10,"Snap Medical Industries, LLC",1,,
3,Jan 4 2023 8:35AM,254238,10,Emerginnova,1,,
4,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17,,
5,Jan 4 2023 8:26AM,254237,20,"HVL, LLC dba Atrium Innovations",1,,
6,Jan 4 2023 8:25AM,254236,22,"NBTY Global, Inc.",1,,
7,Jan 4 2023 7:54AM,254235,19,"AdvaGen Pharma, Ltd",1,,
8,Jan 3 2023 4:37PM,254230,15,"Person & Covey, Inc.",99,,
9,Jan 3 2023 4:22PM,254227,15,Lonza,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:46AM,254242,10,Bio-PRF,1,,
1,Jan 4 2023 8:43AM,254240,10,"Methapharm, Inc.",6,,
2,Jan 4 2023 8:37AM,254239,10,"Snap Medical Industries, LLC",1,,
3,Jan 4 2023 8:35AM,254238,10,Emerginnova,1,,
4,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:46AM,254242,10,Bio-PRF,1.0,NaN,NaN
1,Jan 4 2023 8:43AM,254240,10,"Methapharm, Inc.",6.0,NaN,NaN
2,Jan 4 2023 8:37AM,254239,10,"Snap Medical Industries, LLC",1.0,NaN,NaN
3,Jan 4 2023 8:35AM,254238,10,Emerginnova,1.0,NaN,NaN
4,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jan 4 2023 8:46AM,254242,10,Bio-PRF,1.0,0.0,0.0
1,Jan 4 2023 8:43AM,254240,10,"Methapharm, Inc.",6.0,0.0,0.0
2,Jan 4 2023 8:37AM,254239,10,"Snap Medical Industries, LLC",1.0,0.0,0.0
3,Jan 4 2023 8:35AM,254238,10,Emerginnova,1.0,0.0,0.0
4,Jan 4 2023 8:30AM,254196,10,ISDIN Corporation,17.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,4320974,333.0,1.0,1.0
12,254124,0.0,1.0,0.0
15,1525154,166.0,1.0,59.0
19,2796242,78.0,4.0,0.0
20,508386,1.0,1.0,0.0
21,508419,2.0,0.0,0.0
22,254236,1.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,4320974,333.0,1.0,1.0
1,12,254124,0.0,1.0,0.0
2,15,1525154,166.0,1.0,59.0
3,19,2796242,78.0,4.0,0.0
4,20,508386,1.0,1.0,0.0
5,21,508419,2.0,0.0,0.0
6,22,254236,1.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,333.0,1.0,1.0
1,12,0.0,1.0,0.0
2,15,166.0,1.0,59.0
3,19,78.0,4.0,0.0
4,20,1.0,1.0,0.0
5,21,2.0,0.0,0.0
6,22,1.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,333.0
1,12,Released,0.0
2,15,Released,166.0
3,19,Released,78.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,20,21,22
Status,,,,,,,
Completed,1.0,0.0,59.0,0.0,0.0,0.0,0.0
Executing,1.0,1.0,1.0,4.0,1.0,0.0,0.0
Released,333.0,0.0,166.0,78.0,1.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,20,21,22
0,Completed,1.0,0.0,59.0,0.0,0.0,0.0,0.0
1,Executing,1.0,1.0,1.0,4.0,1.0,0.0,0.0
2,Released,333.0,0.0,166.0,78.0,1.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,20,21,22
0,Completed,1.0,0.0,59.0,0.0,0.0,0.0,0.0
1,Executing,1.0,1.0,1.0,4.0,1.0,0.0,0.0
2,Released,333.0,0.0,166.0,78.0,1.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()